# Drowsiness Detection OpenCV




In [1]:
from scipy.spatial import distance
from imutils import face_utils
import time
import imutils
import dlib
import cv2
import socket

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8089))
server.listen(1)

In [2]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [3]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [4]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
cap=cv2.VideoCapture(0)
flag=0
conn, addr = server.accept()
cmnd = conn.recv(5) 
print(cmnd)
while True:
 # The default size of the command packet is 4 bytes
	ret, frame=cap.read() #بيرجع قيمة بوليانية (True/False). 
    #If frame is read correctly, it will be True. So you can check end of the video by checking this return value.
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	subjects = detect(gray, 0) #دورلي عل الوجوه بالصورة (الاطار)
	for subject in subjects: #عم يلقط أكتر من وجه 
		shape = predict(gray, subject)
		shape = face_utils.shape_to_np(shape)#converting to NumPy Array
		leftEye = shape[lStart:lEnd] #مصفوفة من بداية العين اليسار إلى نهايتها 
		rightEye = shape[rStart:rEnd] #مصفوفة من بداية العين اليمين الى نهايتها 
		leftEAR = eye_aspect_ratio(leftEye) #نسبة ارتفاع العين اليسرى 
		rightEAR = eye_aspect_ratio(rightEye) #نسبة ارتفاع العين اليمنى
		ear = (leftEAR + rightEAR) / 2.0 #المتوسط الحسابي لمجموع النسبتين 
		leftEyeHull = cv2.convexHull(leftEye)# يقوم بفحص منحنى لعيوب التحدب وتصحيحها
		rightEyeHull = cv2.convexHull(rightEye)
		if ear < thresh:#اذا كانت نسبة ارتفاع العين اصغر من العتبة يلي هي 0.25
			flag += 1
			if flag >= frame_check:#اذا العداد عدد الفريمات يلي نسبة ارتفاع العين اقل من العتبة هو اقل من عتبة عدد الفريمات 20
				conn.sendall(b'drows')
				print ('drows')
				time.sleep(1)
		else:
			conn.sendall(b'Awake')
			print ('AWake')
			flag = 0   
			time.sleep(1)
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		cv2.destroyAllWindows()
		cap.release()
		break


In [ ]:
#Error 56 occurred at TCP Read in LVClient2011.vi
#LabVIEW: (Hex 0x38) The network operation exceeded the user-specified or system time limit.